Source: https://colab.research.google.com/github/DhavalTaunk08/Transformers_scripts/blob/master/Transformers_multilabel_distilbert.ipynb#scrollTo=StbPlIyKDP9E

In [29]:
import pandas as pd
pd.read_parquet("hf://datasets/data-is-better-together/fineweb-c/dan_Latn/train-00000-of-00001.parquet")

,id,text,educational_value_labels,annotator_ids,problematic_content_label_present,problematic_content_label_agreement,language_names,language_code
0,c293383d-cffc-4244-9e75-1cf1a25eb0cc,"Er du en af de mange, der ikke kan se 6′eren d...","[None, None, None]","[a0585a5c-b72f-4c3a-a2a3-17e8e0b4ea4f, 85ac8d5...",False,1.000000,dan_Latn,dan_Latn
1,8ebfa2e5-fdb4-494a-817a-3a5b0fe09475,Behandlingsområder\nHer kan du læse om regione...,"[None, Minimal]","[a0585a5c-b72f-4c3a-a2a3-17e8e0b4ea4f, 85ac8d5...",False,1.000000,dan_Latn,dan_Latn
2,917f3945-cec8-4cfe-8516-c72351489fa6,Schul Landskabsarkitekter arbejder med have- o...,"[None, Minimal, None]","[95e4f810-6fe7-4bbd-8be6-a11b0d069c11, a0585a5...",False,1.000000,dan_Latn,dan_Latn
3,8defb5be-a075-4709-9d61-db3632f61e13,Nyuddannede hurtigt i arbejde ad mælkevejen - ...,"[Minimal, Minimal]","[a0585a5c-b72f-4c3a-a2a3-17e8e0b4ea4f, 85ac8d5...",False,1.000000,dan_Latn,dan_Latn
4,f71ad46e-cd61-45fb-8ab7-d82bdd5885f1,I onsdags var Kristian og jeg inde og kigge på...,"[None, ❗ Problematic Content ❗, None]","[a0585a5c-b72f-4c3a-a2a3-17e8e0b4ea4f, 9987848...",True,0.333333,dan_Latn,dan_Latn
...,...,...,...,...,...,...,...,...
995,da2d09ad-af5d-4690-8474-72152a61d4f8,Sorte singler datingYozshudal\nFind kærlighede...,"[❗ Problematic Content ❗, ❗ Problematic Conten...","[a0585a5c-b72f-4c3a-a2a3-17e8e0b4ea4f, 85ac8d5...",True,1.000000,dan_Latn,dan_Latn
996,74f77a6f-f30e-48d1-b90a-f1673bd2787a,"Datjng er mange gode grunde til, at kvinder, a...","[❗ Problematic Content ❗, None, ❗ Problematic ...","[a0585a5c-b72f-4c3a-a2a3-17e8e0b4ea4f, b98b014...",True,0.666667,dan_Latn,dan_Latn
997,cd146702-e112-415a-9017-83ebaed7390a,gay aroma thai massage århus porno ældre herre...,"[❗ Problematic Content ❗, ❗ Problematic Conten...","[a0585a5c-b72f-4c3a-a2a3-17e8e0b4ea4f, b98b014...",True,1.000000,dan_Latn,dan_Latn
998,a7d17d2b-90ec-4bc5-b6a7-ceb66c5d76e2,Borgerservice genåbner i dag\nBorgerservice i ...,"[Minimal, Minimal]","[a0585a5c-b72f-4c3a-a2a3-17e8e0b4ea4f, 85ac8d5...",False,1.000000,dan_Latn,dan_Latn


<a id='section01'></a>
### Importing Python Libraries and preparing the environment

At this step we will be importing the libraries and modules needed to run our script. Libraries are:
* Pandas
* Pytorch
* Pytorch Utils for Dataset and Dataloader
* Transformers
* BERT Model and Tokenizer

Followed by that we will preapre the device for GPU execeution. This configuration is needed if you want to leverage on onboard GPU.

*I have included the code for TPU configuration, but commented it out. If you plan to use the TPU, please comment the GPU execution codes and uncomment the TPU ones to install the packages and define the device.*

In [30]:
# Installing the transformers library and additional libraries if looking process

!pip install -q transformers

# Code for TPU packages install
# !curl -q https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
# !python pytorch-xla-env-setup.py --apt-packages libomp5 libopenblas-dev

In [1]:
# Importing stock ML Libraries
import numpy as np
import pandas as pd
from sklearn import metrics
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertModel, BertConfig
from sklearn.model_selection import train_test_split
from collections import Counter
import ast
import matplotlib.pyplot as plt
import json

In [2]:
# # Setting up the device for GPU usage

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

<a id='section02'></a>
### Importing and Pre-Processing the domain data

We will be working with the data and preparing for fine tuning purposes.
*Assuming that the `train.csv` is already downloaded, unzipped and saved in your `data` folder*

* Import the file in a dataframe and give it the headers as per the documentation.
* Taking the values of all the categories and coverting it into a list.
* The list is appened as a new column and other columns are removed

In [3]:
def Most_common_label(label_list):
    label = Counter(label_list).most_common(1)[0][0]
    inx = sort_order[label]  # Most frequent label
    L = [0 for i in range(len(unique_labels))]
    L[inx] = 1
    return L


def Soft_label(label_list):
    numeric_annotations = [sort_order[label] for label in label_list]
    return np.bincount(numeric_annotations, minlength=len(unique_labels)) / len(label_list)

In [9]:
#########
DATASET = pd.read_parquet("hf://datasets/data-is-better-together/fineweb-c/dan_Latn/train-00000-of-00001.parquet")
PROBLEMATIC_CONTENT = False
LABEL_FUNCTION = Most_common_label
ex_Data_path = "fineweb2_data.csv"
Binary_Classification = True 
#########


df = pd.DataFrame()
df["text"] = DATASET["text"]
df["educational_value_labels"] = DATASET["educational_value_labels"]
df["problematic_content_label_present"] = DATASET["problematic_content_label_present"]

extra_df = pd.read_csv(ex_Data_path)
extra_df = extra_df.rename(columns={"label": "educational_value_labels"})
extra_df['educational_value_labels'] = extra_df['educational_value_labels'].apply(ast.literal_eval)
extra_df['problematic_content_label_present'] = False

df = pd.concat([df, extra_df], ignore_index=True)

# REMOVE PROBLEMATIC LABELS FROM DATASET
df = df[df['problematic_content_label_present'] == PROBLEMATIC_CONTENT]

unique_labels = df["educational_value_labels"].explode().unique().tolist()
sort_order = {
    "None": unique_labels.index("None"),
    "Minimal": unique_labels.index("Minimal"),
    "Basic": unique_labels.index("Basic"),
    "Good": unique_labels.index("Good"),
    "Excellent": unique_labels.index("Excellent"),
}

# Convert Multi-Labels to Binary Labels
if Binary_Classification:
    unique_labels = ["None", "Educational"]
    sort_order = {
        "None": 0,
        "Educational": 1
    }
    df["educational_value_labels"] = df["educational_value_labels"].apply(
        lambda x: ["Educational"] if "None" not in x else ["None"])

display(df.sample(5))

# Process Data labels
df["Final_label"] = df["educational_value_labels"].apply(LABEL_FUNCTION)

df.sample(5)

,text,educational_value_labels,problematic_content_label_present
114,I dagene fra onsdag den 12. til og med lørdag ...,[Educational],False
308,Nyheder\n22.08.14 - Tillykke til vinderne af P...,[None],False
466,Lad dit barns kreativitet blomstre\nGiv dit ba...,[None],False
197,I Hedensted Kommune betaler vi for at bruge la...,[Educational],False
662,Efter hovedværket 'De ti skud' i 2004 bedyrede...,[Educational],False


,text,educational_value_labels,problematic_content_label_present,Final_label
513,Her i vores (ufrivillige) træningsløbspause ka...,[None],False,"[1, 0]"
22,For rejsende der gerne vil opleve synet og lyd...,[None],False,"[1, 0]"
295,Baser dig selv i det travle centrum af Milanos...,[Educational],False,"[0, 1]"
794,"Martin Hvidt\nLektor, dr.phil.\nBiografi\nMart...",[None],False,"[1, 0]"
792,Marcelos søn med hattrick i El Derbi\nTil alle...,[None],False,"[1, 0]"


In [10]:
new_df = pd.DataFrame()
new_df["text"] = df["text"]
new_df["labels"] = df["Final_label"]
unique_labels

['None', 'Educational']

<a id='section03'></a>
### Preparing the Dataset and Dataloader

We will start with defining few key variables that will be used later during the training/fine tuning stage.
Followed by creation of CustomDataset class - This defines how the text is pre-processed before sending it to the neural network. We will also define the Dataloader that will feed  the data in batches to the neural network for suitable training and processing.
Dataset and Dataloader are constructs of the PyTorch library for defining and controlling the data pre-processing and its passage to neural network. For further reading into Dataset and Dataloader read the [docs at PyTorch](https://pytorch.org/docs/stable/data.html)

#### *CustomDataset* Dataset Class
- This class is defined to accept the `tokenizer`, `dataframe` and `max_length` as input and generate tokenized output and tags that is used by the BERT model for training.
- We are using the BERT tokenizer to tokenize the data in the `comment_text` column of the dataframe.
- The tokenizer uses the `encode_plus` method to perform tokenization and generate the necessary outputs, namely: `ids`, `attention_mask`, `token_type_ids`
---
- *This is the first difference between the distilbert and bert, where the tokenizer generates the token_type_ids in case of Bert*
---
- To read further into the tokenizer, [refer to this document](https://huggingface.co/transformers/model_doc/bert.html#berttokenizer)
- `targest` is the list of categories labled as `0` or `1` in the dataframe.
- The *CustomDataset* class is used to create 2 datasets, for training and for validation.
- *Training Dataset* is used to fine tune the model: **80% of the original data**
- *Validation Dataset* is used to evaluate the performance of the model. The model has not seen this data during training.

#### Dataloader
- Dataloader is used to for creating training and validation dataloader that load data to the neural network in a defined manner. This is needed because all the data from the dataset cannot be loaded to the memory at once, hence the amount of dataloaded to the memory and then passed to the neural network needs to be controlled.
- This control is achieved using the parameters such as `batch_size` and `max_len`.
- Training and Validation dataloaders are used in the training and validation part of the flow respectively

In [36]:
# Sections of config

# Defining some key variables that will be used later on in the training
MAX_LEN = 128
TRAIN_SIZE = 0.8
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 4
EPOCHS = 100
LEARNING_RATE = 1e-05
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
class MultiLabelDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.text
        self.targets = self.data.labels
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length', # New: replaces deprecated pad_to_max_length (pad_to_max_length=True,)
            truncation=True,      # explicitly set truncation
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [38]:
print(new_df)

                                                   text           labels
0     Er du en af de mange, der ikke kan se 6′eren d...  [1, 0, 0, 0, 0]
1     Behandlingsområder\nHer kan du læse om regione...  [1, 0, 0, 0, 0]
2     Schul Landskabsarkitekter arbejder med have- o...  [1, 0, 0, 0, 0]
3     Nyuddannede hurtigt i arbejde ad mælkevejen - ...  [0, 1, 0, 0, 0]
5     Beliggenhed: Kina > Beijing > Beijing West Rai...  [1, 0, 0, 0, 0]
...                                                 ...              ...
1008  Anden renses for indmad og skylles. Gnides her...  [0, 0, 0, 1, 0]
1009  Lusen er et snyltedyr, dvs. et lille insekt, d...  [0, 0, 0, 0, 1]
1010  \n\nHvad er Ordblindhed (dysleksi)\n|Forfatter...  [0, 0, 0, 0, 1]
1011  skum, masse af gasfyldte bobler med elastiske ...  [0, 0, 0, 1, 0]
1012  genteknologi, betegnelse for en række metoder,...  [0, 0, 0, 0, 1]

[819 rows x 2 columns]


In [39]:
# Creating the dataset and dataloader for the neural network

train_size = TRAIN_SIZE
train_data=new_df.sample(frac=train_size,random_state=200)
test_data=new_df.drop(train_data.index).reset_index(drop=True)
train_data = train_data.reset_index(drop=True)


print("FULL Dataset: {}".format(new_df.shape))
print("TRAIN Dataset: {}".format(train_data.shape))
print("TEST Dataset: {}".format(test_data.shape))

training_set = MultiLabelDataset(train_data, tokenizer, MAX_LEN)
testing_set = MultiLabelDataset(test_data, tokenizer, MAX_LEN)

FULL Dataset: (819, 2)
TRAIN Dataset: (655, 2)
TEST Dataset: (164, 2)


### Check class distribution in datasets

In [40]:
### Check class distribution

def class_distribution(df):
    idx = df['labels'].apply(lambda x: np.argmax(x))

    # Get distribution
    class_counts = idx.value_counts().sort_index()

    print("Class distribution in training set:")
    print(class_counts)

class_distribution(train_data)
class_distribution(test_data)

Class distribution in training set:
labels
0    374
1    215
2     49
3     14
4      3
Name: count, dtype: int64
Class distribution in training set:
labels
0    97
1    45
2    15
3     6
4     1
Name: count, dtype: int64


In [41]:
train_data.sample(5)

,text,labels
53,En helt enkel højtaler til tablets uden behov ...,"[1, 0, 0, 0, 0]"
159,Fuglehus til haven – Find det perfekte foderbr...,"[1, 0, 0, 0, 0]"
262,PSYKIATRI Der er flere grunde til at være glad...,"[0, 1, 0, 0, 0]"
584,FODBOLD: Der er ofte udfordringer for Brabrand...,"[1, 0, 0, 0, 0]"
357,Sådan bliver du dig selv3. april 2011\nLev i n...,"[1, 0, 0, 0, 0]"


In [42]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [43]:
testing_loader

<a id='section04'></a>
### Creating the Neural Network for Fine Tuning

#### Neural Network
 - We will be creating a neural network with the `BERTClass`.
 - This network will have the `Bert` model.  Follwed by a `Droput` and `Linear Layer`. They are added for the purpose of **Regulariaztion** and **Classification** respectively.
 - In the forward loop, there are 2 output from the `BertModel` layer.
 - The second output `output_1` or called the `pooled output` is passed to the `Drop Out layer` and the subsequent output is given to the `Linear layer`.
 - Keep note the number of dimensions for `Linear Layer` is **6** because that is the total number of categories in which we are looking to classify our model.
 - The data will be fed to the `BertClass` as defined in the dataset.
 - Final layer outputs is what will be used to calcuate the loss and to determine the accuracy of models prediction.
 - We will initiate an instance of the network called `model`. This instance will be used for training and then to save the final trained model for future inference.

#### Loss Function and Optimizer
 - The Loss is defined in the next cell as `loss_fn`.
 - As defined above, the loss function used will be a combination of Binary Cross Entropy which is implemented as [BCELogits Loss](https://pytorch.org/docs/stable/nn.html#bcewithlogitsloss) in PyTorch
 - `Optimizer` is defined in the next cell.
 - `Optimizer` is used to update the weights of the neural network to improve its performance.

#### Further Reading
- You can refer to my [Pytorch Tutorials](https://github.com/abhimishra91/pytorch-tutorials) to get an intuition of Loss Function and Optimizer.
- [Pytorch Documentation for Loss Function](https://pytorch.org/docs/stable/nn.html#loss-functions)
- [Pytorch Documentation for Optimizer](https://pytorch.org/docs/stable/optim.html)
- Refer to the links provided on the top of the notebook to read more about `BertModel`.

In [44]:
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model.

class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.l1 = transformers.BertModel.from_pretrained('bert-base-uncased')
        self.l2 = torch.nn.Dropout(0.3)
        #Change the secound val to the number of classes !!!!!!!!
        self.l3 = torch.nn.Linear(768, len(unique_labels))

    def forward(self, ids, mask, token_type_ids):
        _, output_1= self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids, return_dict=False)
        output_2 = self.l2(output_1)
        output = self.l3(output_2)
        return output

model = BERTClass()
model.to(device)

BERTClass(
  (l1): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affin

### Loss, accuracy, (weighted loss)

In [ ]:
# Convert one-hot encoded labels to class indices
class_indices = [label.index(1) for label in train_data['labels']]
class_counts = torch.bincount(torch.tensor(class_indices))
class_weights = 1.0 / class_counts.float()  # inverse frequency
class_weights = class_weights / class_weights.sum()  # normalization

# Define loss_fn with weights
weighted_loss_fn = torch.nn.CrossEntropyLoss(weight=class_weights.to(device))


def loss_fn(outputs, targets):
    # return torch.nn.CrossEntropyLoss()(outputs, targets)
    
    target_indices = torch.argmax(targets, dim=1)  # [batch_size]
    return weighted_loss_fn(outputs, target_indices)

def accuracyTest(outputs, targets):
    outputs = outputs.to(device, dtype=torch.int)
    targets = targets.to(device, dtype=torch.int)

    correct = torch.all(outputs == targets, dim=1)

    correct_num = correct.sum().item()

    accuracy = correct_num / len(targets)

    return accuracy

In [46]:
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [ ]:
train_accuracies_pr_epoch = []
test_accuracies_pr_epoch = []
train_losses = []
confusion_matrix_pr_epoch = []

def train(epoch):
    model.train()
    total_loss = 0  # Track total loss for the epoch
    train_accuracies = []

    for batch_idx,data in enumerate(training_loader, 0):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)
        probs = torch.softmax(outputs, dim=1) # SOFTMAX (dim=1) or SIGMOID (), depends on how we interpret the multiclass
    
        max_indices = torch.argmax(probs, dim=1)
        preds = torch.zeros_like(probs)
        preds.scatter_(1, max_indices.unsqueeze(1), 1)

        # print(preds)
        # print(targets)
        batch_train_accuracy = accuracyTest(preds, targets)
        train_accuracies.append(batch_train_accuracy)

        print(f"Batch {batch_idx} Train Accuracy:", batch_train_accuracy)
        
        loss = loss_fn(outputs, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

        # Print loss every 10 batches (adjust as needed)
        if batch_idx % 10 == 0:
            print(f'Epoch: {epoch}, Batch: {batch_idx}, Loss: {loss.item()}')

    
    train_accuracies_pr_epoch.append(np.mean(train_accuracies)) # Save avrage accuracies for the epoch
    avg_train_loss = total_loss / len(training_loader)
    train_losses.append(avg_train_loss)  # Save average loss for the epoch
    print(f'Epoch {epoch} Average Training Loss: {avg_train_loss}, Avrage Training Accuraccy: {np.mean(train_accuracies)}')


def validation(epoch):
    model.eval()
    confusion_matrix = np.zeros((len(unique_labels), len(unique_labels)))

    test_acc = []
    with torch.no_grad():
        for _, data in enumerate(testing_loader, 0):
            ids = data['ids'].to(device, dtype=torch.long)
            mask = data['mask'].to(device, dtype=torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
            targets = data['targets'].to(device, dtype=torch.float)
            outputs = model(ids, mask, token_type_ids)

            probs = torch.softmax(outputs, dim=1) # SOFTMAX (dim=1) or SIGMOID (), depends on how we interpret the multiclass
        
            max_indices = torch.argmax(probs, dim=1)
            preds = torch.zeros_like(probs)
            preds.scatter_(1, max_indices.unsqueeze(1), 1)

            print(preds)
            print(targets)

            # Print confusion matrix for each sample
            for i, (target, pred) in enumerate(zip(targets, preds)):
                true_class = target.argmax().item()
                predicted_class = pred.argmax().item()
                
                # Print per-sample prediction
                print(f"Sample {i}: true class = {true_class}, predicted = {predicted_class}")
                # Update confusion matrix
                confusion_matrix[true_class, predicted_class] += 1
            

            batch_test_acc = accuracyTest(preds, targets)
            test_acc.append(batch_test_acc)
            
    # Print confusion matrix for the epoch        
    print(confusion_matrix)
    confusion_matrix_pr_epoch.append(confusion_matrix)  # Save confusion matrix for the epoch
    # Calculate metrics
    test_accuracies_pr_epoch.append(np.mean(test_acc)) # Save avrage accuracies for the epoch
    print(f'Epoch {epoch}, Avrage Test Accuraccy: {np.mean(test_acc)}')



In [52]:
### Epoch Loop

for epoch in range(EPOCHS):
    train(epoch)
    validation(epoch)


Batch 0 Train Accuracy: 0.25
Epoch: 0, Batch: 0, Loss: 0.47275489568710327
Batch 1 Train Accuracy: 0.25
Batch 2 Train Accuracy: 0.75
Batch 3 Train Accuracy: 0.5
Batch 4 Train Accuracy: 0.25
Batch 5 Train Accuracy: 0.5
Batch 6 Train Accuracy: 0.25
Batch 7 Train Accuracy: 0.0
Batch 8 Train Accuracy: 0.75
Batch 9 Train Accuracy: 0.5
Batch 10 Train Accuracy: 1.0
Epoch: 0, Batch: 10, Loss: 0.2851070463657379
Batch 11 Train Accuracy: 0.5
Batch 12 Train Accuracy: 0.75
Batch 13 Train Accuracy: 0.25
Batch 14 Train Accuracy: 1.0
Batch 15 Train Accuracy: 0.5
Batch 16 Train Accuracy: 0.5
Batch 17 Train Accuracy: 1.0
Batch 18 Train Accuracy: 0.75
Batch 19 Train Accuracy: 0.75
Batch 20 Train Accuracy: 0.5
Epoch: 0, Batch: 20, Loss: 0.4490510821342468
Batch 21 Train Accuracy: 0.75
Batch 22 Train Accuracy: 0.5
Batch 23 Train Accuracy: 0.75
Batch 24 Train Accuracy: 0.5
Batch 25 Train Accuracy: 0.5
Batch 26 Train Accuracy: 0.5
Batch 27 Train Accuracy: 0.5
Batch 28 Train Accuracy: 0.0
Batch 29 Train Accu

KeyboardInterrupt: 

### Metrics

In [ ]:
def tp(multi_class_confusion_matrix):
    return np.diag(multi_class_confusion_matrix)

def fp(multi_class_confusion_matrix):
    return np.sum(multi_class_confusion_matrix, axis=0) - tp(multi_class_confusion_matrix)

def fn(multi_class_confusion_matrix):
    return np.sum(multi_class_confusion_matrix, axis=1) - tp(multi_class_confusion_matrix)

def tn(multi_class_confusion_matrix):
    total = np.sum(multi_class_confusion_matrix)
    return total - (tp(multi_class_confusion_matrix) + fp(multi_class_confusion_matrix) + fn(multi_class_confusion_matrix))

def precision(tp,fp):
    return tp / (tp + fp) if (tp + fp).all() else np.nan

def recall(tp,fn):
    return tp / (tp + fn) if (tp + fn).all() else np.nan

def f1_score(precision, recall):
    return 2 * (precision * recall) / (precision + recall) if (precision + recall).all() else np.nan

def cm_metrics(cm):
    tp_values = tp(cm)
    fp_values = fp(cm)
    fn_values = fn(cm)
    tn_values = tn(cm)

    class_metrics = {}

    for i in range(len(cm)):
        precision_values = precision(tp_values[i], fp_values[i])
        recall_values = recall(tp_values[i], fn_values[i])
        f1_values = f1_score(precision_values, recall_values)

        class_metrics[i] = {
            "TP": tp_values[i],
            "FP": fp_values[i],
            "FN": fn_values[i],
            "TN": tn_values[i],
            "Precision": precision_values,
            "Recall": recall_values,
            "F1 Score": f1_values
        }

    return class_metrics

In [ ]:
### Plots & Metrics

all_epoch_metrics = []
for epoch, cm in enumerate(confusion_matrix_pr_epoch):  # confusion_matrices = list of matrices per epoch
    class_metrics = cm_metrics(cm)
    
    # Attach epoch info
    epoch_entry = {
        "epoch": epoch,
        "metrics": class_metrics  # class_metrics is already in {class_id: {...}} format
    }
    
    all_epoch_metrics.append(epoch_entry)

with open("epoch_metrics.json", "w") as f:
    json.dump(all_epoch_metrics, f, indent=2)

epochs = list(range(EPOCHS))

plt.plot(epochs, train_accuracies_pr_epoch, label='Train Accuracy')
plt.plot(epochs, test_accuracies_pr_epoch, label='Validation Accuracy')
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("Train vs Validation Accuracy")
plt.legend()
plt.grid(True)

plt.savefig("learningcurve.png")

plt.show()